In [1]:
import pandas as pd
import numpy as np

In [3]:
df = pd.read_csv("DraftKingsdata.csv")
df.head()

,Position,Name,Salary,GameInfo,AvgPointsPerGame,teamAbbrev
0,PG,Russell Westbrook,11600,NO@OKC 02/02/2018 09:00PM ET,57.172,OKC
1,PF/C,DeMarcus Cousins,11400,NO@OKC 02/02/2018 09:00PM ET,0.000,NO
2,PF/C,Anthony Davis,11400,NO@OKC 02/02/2018 09:00PM ET,50.466,NO
3,SF/PF,Giannis Antetokounmpo,10800,NY@MIL 02/02/2018 08:00PM ET,53.656,MIL
4,PG/SG,Stephen Curry,10600,GS@SAC 02/02/2018 10:30PM ET,48.111,GS


In [29]:
uniques = set()
for row in df.Position:
    for item in row.split("/"):
        uniques.add(item)
positions_list = list(uniques)
positions_list

['SG', 'C', 'SF', 'PF', 'PG']

In [30]:
for position in positions_list[::-1]:
    df[position] = 0
df.columns

Index(['Position', 'Name', 'Salary', 'GameInfo', 'AvgPointsPerGame',
       'teamAbbrev', 'SG', 'C', 'SF', 'PF', 'PG'],
      dtype='object')

In [31]:
for i in range(len(df.Position)):
    for item in df.Position[i].split("/"):
        df.loc[i,item] = 1
df.head()

,Position,Name,Salary,GameInfo,AvgPointsPerGame,teamAbbrev,SG,C,SF,PF,PG
0,PG,Russell Westbrook,11600,NO@OKC 02/02/2018 09:00PM ET,57.172,OKC,0,0,0,0,1
1,PF/C,DeMarcus Cousins,11400,NO@OKC 02/02/2018 09:00PM ET,0.000,NO,0,1,0,1,0
2,PF/C,Anthony Davis,11400,NO@OKC 02/02/2018 09:00PM ET,50.466,NO,0,1,0,1,0
3,SF/PF,Giannis Antetokounmpo,10800,NY@MIL 02/02/2018 08:00PM ET,53.656,MIL,0,0,1,1,0
4,PG/SG,Stephen Curry,10600,GS@SAC 02/02/2018 10:30PM ET,48.111,GS,1,0,0,0,1


In [67]:
P = df.iloc[:,-5:]
P.head()

df.to_csv("DraftKingsdata_Cleaner.csv", index=False)

In [64]:
import os
import pyomo.environ as pyo
from pyomo.environ import *

from pyomo.opt import SolverFactory

opt = pyo.SolverFactory('glpk')

model = ConcreteModel()

Names = df.Name.tolist()
Salaries = df.Salary.tolist()
AvgPoints = df.AvgPointsPerGame.tolist()
P = df.iloc[:,-5:].values
budget = 50000

N = len(Names)
model.x = Var(range(N),
               within=Binary)
model.g1 = Var(within=Binary)
model.h1 = Var(within=Binary)

model.obj = Objective(expr = pyo.summation(AvgPoints, model.x), 
                      sense=maximize)

In [65]:
#import pyomo.environ as aml
model.constraints = ConstraintList()

# Valid player constraints
#aml.inequality(1+model.g1, pyo.summation(P[:,4],model.x), 2+model.g1)
model.constraints.add( (1+model.g1) <= pyo.summation(P[:,4],model.x)<=(2+model.g1) )
model.constraints.add( (1+(1-model.g1)) <= pyo.summation(P[:,0],model.x)<=(2+(1-model.g1)) )
model.constraints.add( (1+model.h1) <= pyo.summation(P[:,2],model.x)<=(2+model.h1) )
model.constraints.add( (1+(1-model.h1)) <= pyo.summation(P[:,3],model.x)<=(2+(1-model.h1)) )
model.constraints.add( 1<= pyo.summation(P[:,1],model.x)<=2 )

# Salary/budget
model.constraints.add( pyo.summation(Salaries,model.x)<=budget )

    function to express ranged inequality expressions.
    function to express ranged inequality expressions.
    function to express ranged inequality expressions.
    function to express ranged inequality expressions.
    function to express ranged inequality expressions.


In [66]:
# Create a model instance and optimize
instance = model.create_instance()
results = opt.solve(instance)
instance.display()
print ("\n===== results",i)
results = opt.solve(instance)
instance.display()

    model; returning a clone of the current model instance.
ERROR: evaluating object as numeric value: g1
        (object: <class 'pyomo.core.base.var.SimpleVar'>)
    No value for uninitialized NumericValue object g1


ValueError: No value for uninitialized NumericValue object g1